In [117]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets
from sklearn.metrics import classification_report, confusion_matrix, r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [118]:
import os
import sys
sys.path.insert(0, '../modules')
import mlp


# Датасет Titanic

## Подготовка данных

Импортируем датасет

In [119]:
data = pd.read_csv('../data/titanic.csv', sep = ',')

In [120]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [121]:
data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
data.fillna(data.mean(), inplace=True)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.000000,1,0,7.2500,S
1,1,1,female,38.000000,1,0,71.2833,C
2,1,3,female,26.000000,0,0,7.9250,S
3,1,1,female,35.000000,1,0,53.1000,S
4,0,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S
887,1,1,female,19.000000,0,0,30.0000,S
888,0,3,female,29.699118,1,2,23.4500,S
889,1,1,male,26.000000,0,0,30.0000,C


In [122]:
data = pd.get_dummies(data=data, prefix='c', columns=['Sex', 'Embarked'])
data

,Survived,Pclass,Age,SibSp,Parch,Fare,c_female,c_male,c_C,c_Q,c_S
0,0,3,22.000000,1,0,7.2500,0,1,0,0,1
1,1,1,38.000000,1,0,71.2833,1,0,1,0,0
2,1,3,26.000000,0,0,7.9250,1,0,0,0,1
3,1,1,35.000000,1,0,53.1000,1,0,0,0,1
4,0,3,35.000000,0,0,8.0500,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.0000,0,1,0,0,1
887,1,1,19.000000,0,0,30.0000,1,0,0,0,1
888,0,3,29.699118,1,2,23.4500,1,0,0,0,1
889,1,1,26.000000,0,0,30.0000,0,1,1,0,0


In [123]:
y = data[['Survived']]
X = data[data.columns[1:data.shape[1]]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

## Классификация с помощью модели MLP

In [125]:
model = mlp.MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(15, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative),
                                                                      (15, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train.values, y_train.values, epochs=600, learning_rate=0.25)

In [126]:
y_pred = []
for i in range(len(X_test.values)):
    prediction = model.predict(X_test.values[i])[0]
    if prediction > 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [127]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[83 27]
 [28 41]]
              precision    recall  f1-score   support

           0       0.75      0.75      0.75       110
           1       0.60      0.59      0.60        69

    accuracy                           0.69       179
   macro avg       0.68      0.67      0.67       179
weighted avg       0.69      0.69      0.69       179



# Датасет Iris

## Подготовка данных

 Импортируем датасет

In [128]:
iris = datasets.load_iris()

In [129]:
X = iris.data
X

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [145]:
y = iris.target
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [146]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [147]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

##  Классификация с помощью модели MLP

In [148]:
model = mlp.MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative),
                                                                      (3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)],
                outputs_count_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train, y_train, epochs=300, learning_rate=0.15)

In [149]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(np.argmax(model.predict(X_test[i])))

In [150]:
 print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 0  0 10]
 [ 1  4  5]
 [ 0  0 10]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       1.00      0.40      0.57        10
           2       0.40      1.00      0.57        10

    accuracy                           0.47        30
   macro avg       0.47      0.47      0.38        30
weighted avg       0.47      0.47      0.38        30



# Датасет Alizadeh

## Подготовка данных

 Импортируем датасет

In [168]:
df = pd.read_table("../data/classification/alizadeh_preprocessed.csv", sep=',')

In [169]:
from sklearn.preprocessing import LabelEncoder

In [170]:
df

,Age,Weight,Length,Sex,BMI,DM,HTN,Current Smoker,EX-Smoker,FH,...,Lymph,Neut,PLT,EF-TTE,Region RWMA,VHD,LAD,LCX,RCA,Cath
0,53,90,175,1,29.387755,0,1,1,0,0,...,39,52,261,50,0,2,1,0,1,0
1,67,70,157,2,28.398718,0,1,0,0,0,...,38,55,165,40,4,2,1,1,0,0
2,54,54,164,1,20.077335,0,0,1,0,0,...,38,60,230,40,2,0,1,0,0,0
3,66,67,158,2,26.838648,0,1,0,0,0,...,18,72,742,55,0,3,0,0,0,1
4,50,87,153,2,37.165193,0,1,0,0,0,...,55,39,274,50,0,3,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,58,84,168,1,29.761905,0,0,0,0,0,...,34,58,251,45,0,2,1,1,1,0
299,55,64,152,2,27.700831,0,0,0,0,0,...,16,80,377,40,0,0,0,0,0,1
300,48,77,160,2,30.078125,0,1,0,0,1,...,35,55,279,55,0,2,0,0,0,1
301,57,90,159,2,35.599858,1,0,0,0,0,...,48,40,208,55,0,2,0,0,0,1


In [171]:
X = df.iloc[:, :26].values
y = df.iloc[:,27].values
print (df.shape, X.shape, y.shape)

(303, 59) (303, 26) (303,)


In [172]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [173]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Модель MLP

In [174]:
model = mlp.MLP(inputs_count=X_train.shape[1], hidden_layers_counts_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative),
                                                                      (3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)],
                outputs_count_with_act_funcs=[(3, mlp.ActivationFunctions.sigmoid, mlp.ActivationFunctions.sigmoid_derivative)])\
    .fit(X_train, y_train, epochs=300, learning_rate=0.15)

In [175]:
y_pred = []
for i in range(len(X_test)):
    y_pred.append(np.argmax(model.predict(X_test[i])))

In [176]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 4 39]
 [ 4 14]]
              precision    recall  f1-score   support

           0       0.50      0.09      0.16        43
           1       0.26      0.78      0.39        18

    accuracy                           0.30        61
   macro avg       0.38      0.44      0.28        61
weighted avg       0.43      0.30      0.23        61

